In [2]:
from flask import Flask,render_template,Response
import cv2
# from flask_ngrok import run_with_ngrok

In [3]:
app = Flask(__name__)
# run_with_ngrok(app)

camera = cv2.VideoCapture(0)

def generateFrames():
    while True:
        success,frame = camera.read() ##returns 2 parameters boolean(if it is recognise a frame means) and the frame
        if not success:
            break;
        else:
            face_cascade = cv2.CascadeClassifier('./Haarcascades/haarcascade_frontalface_default.xml')
            eye_cascade = cv2.CascadeClassifier('./Haarcascades/haarcascade_eye.xml')
            faces = face_cascade.detectMultiScale(frame, 1.1,7)
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            for (x,y,w,h) in faces:
                cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255),2)
                grayColour = gray[y:y+h, x:x+w]
                roi_color = frame[y:y+h, x:x+w]
                eyes = eye_cascade.detectMultiScale(grayColour, 1.1,3)
                for (ex,ey,ew,eh) in eyes:
                    cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(255, 0, 0),2)
                    # converting to gray  
                    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
            ans,buffer = cv2.imencode(".jpg",frame);
            frame = buffer.tobytes()
            # return should not be used 
            # because it will only read one time
            yield(b'--frame\r\n'
                b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')#pass as hardcoded string

@app.route('/')
def index():
    return render_template("index.html")

@app.route('/video')
def video():
   return Response(generateFrames(),mimetype='multipart/x-mixed-replace; boundary=frame')

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [25/Aug/2021 18:09:09] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Aug/2021 18:09:11] "GET /video HTTP/1.1" 200 -
